Meeting notes:

##### Make diag.bib update code (notebook)
- Compare titles of bib and semantic scholar
- Match
- If match:
    - Add semantic scholar ID to diag bib
    - Update gscites field from bib entry
- If no match:
    - Check best matches, if DOI and Arxiv --> chose DOI 
    - Define right fields (look at code James) + citations + semantic scolar ID 
    - Add 'automatically added' field so we can trace back automation errors more easilty
    - Add to bib   
    
##### Make sript that checks ss ids and adds new ones to diag bib
    
    
##### Make script that can be used by DIAG members
- Add doi/arxiv
- if DOI check if Arxiv exists, if so --> remove Arxiv
- Sort right fields from DOI --> add fields
- Search for semantic scholar ID
- Print fields so user can check it
- Add to bib
- Manual commit push

##### Make script that checks all semantic scholar IDs and updates gscites
- github action overnight?

##### Note: 
- requests limit, 100 requests per 5 mins for ss 
    - 100 papers at once = 1 request
- google doc: 
    - https://docs.google.com/document/d/1uFyh5Z-wieksq_4Bf8Br7wHpN-buHECunS0VurYnlvg/edit


# Libraries

In [1]:
import os
import numpy as np
import pandas as pd

import requests
# from bibreader import parse_bibtex_file, get_bib_blocks
from bib_handling_code.processbib import read_bibfile
from bib_handling_code.processbib import save_to_file
from difflib import SequenceMatcher

# Example from url giving 100 papers from one of Bram's semantic scolar IDs

In [2]:
staff_id = 123637526 #8038506
staff_ids = [123637526]

In [3]:
url=f'https://api.semanticscholar.org/graph/v1/author/{staff_id}/papers?fields=year,title,authors,externalIds,citationCount&limit=100'

r = requests.get(url)
ss_staff_data = r.json()['data']

In [4]:
for i in ss_staff_data:
    if i['paperId'] == 'c70d5169b5c69171cc2ed3ec40eebda0630988d9':
        display(i)

# Function to get data from DOI/ArXiv

In [5]:
def get_doi_csl(doi):
    response = requests.get(
        f"https://doi.org/{doi}",
        headers={"Accept": "application/vnd.citationstyles.csl+json"},
        timeout=5,
    )
    response.raise_for_status()
    return response.json()

## Example

In [6]:
get_doi_csl('10.48550/arXiv.2302.01738').keys()

dict_keys(['type', 'id', 'categories', 'author', 'issued', 'abstract', 'DOI', 'publisher', 'title', 'URL', 'copyright', 'version'])

In [7]:
ss_staff_data[2]['externalIds']

{'DOI': '10.1007/s00330-023-09826-3',
 'CorpusId': 259200504,
 'PubMed': '37338552'}

In [8]:
staff_id = 8038506
url=f'https://api.semanticscholar.org/graph/v1/author/{staff_id}/papers?fields=year,title,authors,externalIds,citationCount&limit=100'

r = requests.get(url)
ss_staff_data = r.json()['data']

In [9]:
found_dois = []

In [10]:
found_arxiv_doi = False
found_regular_doi = False

In [11]:
#ArXiv
for ss_staff_entry in ss_staff_data:
    if 'DOI' in ss_staff_entry['externalIds']:
        if 'arXiv' in ss_staff_entry['externalIds']['DOI']:
            arxiv_doi = ss_staff_entry['externalIds']['DOI']
            found_arxiv_doi = True
    if found_arxiv_doi:
        break
#DOI
for ss_staff_entry in ss_staff_data:
    if 'DOI' in ss_staff_entry['externalIds']:
        if 'arXiv' not in ss_staff_entry['externalIds']['DOI']:
            regular_doi = ss_staff_entry['externalIds']['DOI']
            
            found_regular_doi = True
    if found_regular_doi:
        break

print(arxiv_doi)
print(regular_doi)

10.48550/arXiv.2302.01790
10.1038/s41598-023-40116-6


In [12]:
arxiv_doi_data = get_doi_csl(arxiv_doi)
regular_doi_data = get_doi_csl(regular_doi)

In [ ]:
for ss in ss_staff_data:
    if 'type' in ss:
        print(ss['type'])

In [17]:
[print(key) for key in arxiv_doi_data.keys()];

type
id
categories
author
issued
abstract
DOI
publisher
title
URL
copyright
version


In [18]:
arxiv_doi_data

{'type': 'article',
 'id': 'https://doi.org/10.48550/arxiv.2302.01790',
 'categories': ['Computer Vision and Pattern Recognition (cs.CV)',
  'FOS: Computer and information sciences',
  'FOS: Computer and information sciences'],
 'author': [{'family': 'Reinke', 'given': 'Annika'},
  {'family': 'Tizabi', 'given': 'Minu D.'},
  {'family': 'Baumgartner', 'given': 'Michael'},
  {'family': 'Eisenmann', 'given': 'Matthias'},
  {'family': 'Heckmann-Nötzel', 'given': 'Doreen'},
  {'family': 'Kavur', 'given': 'A. Emre'},
  {'family': 'Rädsch', 'given': 'Tim'},
  {'family': 'Sudre', 'given': 'Carole H.'},
  {'family': 'Acion', 'given': 'Laura'},
  {'family': 'Antonelli', 'given': 'Michela'},
  {'family': 'Arbel', 'given': 'Tal'},
  {'family': 'Bakas', 'given': 'Spyridon'},
  {'family': 'Benis', 'given': 'Arriel'},
  {'family': 'Blaschko', 'given': 'Matthew'},
  {'family': 'Büttner', 'given': 'Florian'},
  {'family': 'Cardoso', 'given': 'M. Jorge'},
  {'family': 'Cheplygina', 'given': 'Veronika'},

In [19]:
[print(key) for key in regular_doi_data.keys()];

indexed
reference-count
publisher
issue
license
funder
content-domain
abstract
DOI
type
created
update-policy
source
is-referenced-by-count
title
prefix
volume
author
member
published-online
reference
container-title
original-title
language
link
deposited
score
resource
subtitle
short-title
issued
references-count
journal-issue
alternative-id
URL
relation
ISSN
subject
container-title-short
published
assertion
article-number


In [20]:
regular_doi_data

{'indexed': {'date-parts': [[2023, 8, 30]],
  'date-time': '2023-08-30T09:41:19Z',
  'timestamp': 1693388479090},
 'reference-count': 19,
 'publisher': 'Springer Science and Business Media LLC',
 'issue': '1',
 'license': [{'start': {'date-parts': [[2023, 8, 29]],
    'date-time': '2023-08-29T00:00:00Z',
    'timestamp': 1693267200000},
   'content-version': 'tdm',
   'delay-in-days': 0,
   'URL': 'https://creativecommons.org/licenses/by/4.0'},
  {'start': {'date-parts': [[2023, 8, 29]],
    'date-time': '2023-08-29T00:00:00Z',
    'timestamp': 1693267200000},
   'content-version': 'vor',
   'delay-in-days': 0,
   'URL': 'https://creativecommons.org/licenses/by/4.0'}],
 'funder': [{'name': 'The Dutch Lung Foundation', 'award': ['5.1.17.171']}],
 'content-domain': {'domain': ['link.springer.com'],
  'crossmark-restriction': False},
 'abstract': '<jats:title>Abstract</jats:title><jats:p>Accurate identification of emphysema subtypes and severity is crucial for effective management of COPD

---
---
---

#####################################
#  FINAL
#####################################

---
---
---


# Import diag.bib

In [21]:
path_diag_bib = os.path.join('..', 'diag.bib')
diag_bib_raw = read_bibfile(None, path_diag_bib) # I changed the code in such a way that IF I give a second argument, it uses the second argument as a full path

In [22]:
len(diag_bib_raw)

2833

In [23]:
diag_bib_raw[-1].fields

{'author': '{Leon-Ferre, Roberto A. and Carter, Jodi M. and Zahrieh, David and Sinnwell, Jason P. and Salgado, Roberto and Suman, Vera and Hillman, David and Boughey, Judy C. and Kalari, Krishna R. and Couch, Fergus J. and Ingle, James N. and Balkenkohl, Maschenka and Ciompi, Francesco and van der Laak, Jeroen and Goetz, Matthew P.}',
 'title': '{Abstract P2-11-34: Mitotic spindle hotspot counting using deep learning networks is highly associated with clinical outcomes in patients with early-stage triple-negative breast cancer who did not receive systemic therapy}',
 'doi': '{10.1158/1538-7445.sabcs22-p2-11-34}',
 'year': '{2023}',
 'abstract': '{Abstract\n                Background: Triple-negative breast cancers (TNBC) exhibit high rates of recurrence and mortality. However, recent studies suggest that a subset of patients (pts) with early-stage TNBC enriched in tumor-infiltrating lymphocytes (TILs) have excellent clinical outcomes even in the absence of systemic therapy. Additional 

In [24]:
bib_data = []
for bib_entry in diag_bib_raw:
    bibkey = bib_entry.key
    bib_type = bib_entry.type
    
    bib_authors = None
    bib_title = None
    bib_doi = None
    bib_gscites = None
    bib_journal = None
    bib_year = None
    
    if 'author' in bib_entry.fields:
        bib_authors = bib_entry.fields['author'].strip('{ }')
    if 'title' in bib_entry.fields:
        bib_title = bib_entry.fields['title'].strip('{ }')
    if 'doi' in bib_entry.fields:
        bib_doi = bib_entry.fields['doi'].strip('{ }')
    if 'gscites' in bib_entry.fields:
        bib_gscites = bib_entry.fields['gscites'].strip('{ }')
    if 'journal' in bib_entry.fields:
        bib_journal = bib_entry.fields['journal'].strip('{ }')
    if 'year' in bib_entry.fields:
        bib_year = bib_entry.fields['year'].strip('{ }')
    
    bib_data.append([bibkey, bib_type, bib_title, bib_authors, bib_doi, bib_gscites, bib_journal, bib_year])
bib_columns = ['bibkey', 'type', 'title', 'authors', 'doi', 'gs_citations', 'journal', 'year']
df_bib_data = pd.DataFrame(bib_data, columns=bib_columns)

In [25]:
df_bib_data.to_csv('script_data/temp_diag_bib_summary.csv', index=False)

In [26]:
df_bib_data.tail()

,bibkey,type,title,authors,doi,gs_citations,journal,year
2828,Meno23,article,Modelling digital health data: The ExaMode ont...,"Menotti, Laura and Silvello, Gianmaria and Atz...",10.1016/j.jpi.2023.100332,None,Journal of Pathology Informatics,2023
2829,Page23,article,Spatial analyses of immune cell infiltration i...,"Page, David B and Broeckx, Glenn and Jahangir,...",10.1002/path.6165,None,The Journal of Pathology,2023
2830,Aubr23,book,Abstract: the MIDOG Challenge 2021,"Aubreville, Marc and Stathonikos, Nikolas and ...",10.1007/978-3-658-41657-7_26,None,"Bildverarbeitung fur die Medizin, Workshop",2023
2831,Bokh23b,article,Fully Automated Tumor Bud Assessment in Hemato...,"Bokhorst, John-Melle and Ciompi, Francesco and...",10.1016/j.modpat.2023.100233,None,Modern Pathology,2023
2832,Leon23,conference,Abstract P2-11-34: Mitotic spindle hotspot cou...,"Leon-Ferre, Roberto A. and Carter, Jodi M. and...",10.1158/1538-7445.sabcs22-p2-11-34,None,Cancer Research,2023


Used for title matching

In [27]:
df_bib_data_with_titles = df_bib_data[df_bib_data['title'].notna()]
df_bib_data_with_titles.head()

,bibkey,type,title,authors,doi,gs_citations,journal,year
1069,Abas05,article,Analysis of regularity in the {EEG} background...,D. Ab\'{a}solo and R. Hornero and P. Espino an...,10.1016/j.clinph.2005.04.001,233,CLINNP,2005
1070,Genu22,conference,Imaging tumor-infiltrating CD8 (+) T-cells in ...,E. A. J. van Genugten and B. Piet and G. Schre...,None,None,None,2022
1071,Abas05a,inproceedings,EEG} background activity analysis in {A}lzheim...,D. Ab\'{a}solo and C. G\'{o}mez and J. Poza an...,None,3,None,2005
1072,Abel19,article,"Computational pathology definitions, best prac...","Abels, Esther and Pantanowitz, Liron and Aeffn...",10.1002/path.5331,24,JPAT,2019
1073,Abra08a,article,Evaluation of a system for automatic detection...,M. D. Abr\`amoff and M. Niemeijer and M. S. A....,10.2337/dc07-1312,287,DC,2008


# Gather ss and bib data for all staff member IDs

In [28]:
staff_id_dict = {'Bram van Ginneken': [8038506, 123637526],
'Francesco Ciompi': [143613202],
'Alessa Hering': [153744566],
'Henjan Huisman': [34754023],
'Colin Jacobs': [2895994],
'Peter Koopmans': [34726383],
'Jeroen van der Laak': [145441238, 145388932],
'Geert Litjens': [145959882],
'James Meakin': [4960344],
'Keelin Murphy': [35730362],
'Ajay Patel': [2109170880, 2116215861],
'Cornelia Schaefer-Prokop': [1419819133, 1445069528, 1400632685],
'Matthieu Rutten': [2074975080, 2156546],
'Jos Thannhauser': [5752941],
"Bram Platel" : [1798137], 
"Nico Karssemeijer" : [1745574], 
"Clarisa Sanchez" : [144085811, 32187701], 
"Nikolas Lessman" : [2913408], 
"Jonas Teuwen" : [32649341, 119024451], 
"Rashindra Manniesing" : [2657081]}

In [29]:
staff_year_dict = {
'Bram van Ginneken':  {'start' : 1996, 'end': 9999},
'Francesco Ciompi':  {'start' : 2013, 'end': 9999},
'Alessa Hering':  {'start' : 2018, 'end': 9999},
'Henjan Huisman':  {'start' : 1992, 'end': 9999},
'Colin Jacobs':  {'start' : 2010, 'end': 9999},
'Peter Koopmans':  {'start' : 2022, 'end': 9999},
'Jeroen van der Laak':  {'start' : 1991, 'end': 9999},
'Geert Litjens':  {'start' : 2016, 'end': 9999},
'James Meakin':  {'start' : 2017, 'end': 9999},
'Keelin Murphy':  {'start' : 2018, 'end': 9999},
'Ajay Patel':  {'start' : 2015, 'end': 9999},
'Cornelia Schaefer-Prokop':  {'start' : 2010, 'end': 9999},
'Matthieu Rutten':  {'start' : 2019, 'end': 9999},
'Jos Thannhauser': {'start' : 2022, 'end': 9999},
"Bram Platel" : {'start' : 2010,  'end' : 2019},
"Nico Karssemeijer" : {'start' : 1989, 'end' : 2022}, 
"Clarisa Sanchez" : {'start' : 2008, 'end' : 2021}, 
"Nikolas Lessman" : {'start' : 2019, 'end' : 2022}, 
"Jonas Teuwen" : {'start' : 2017, 'end' : 2020}, 
"Rashindra Manniesing" : {'start' : 2010, 'end' : 2021}
}

In [30]:
staff_dict = {key: {'ids': staff_id_dict[key], 'years': staff_year_dict[key]} for key in staff_id_dict}
staff_dict

{'Bram van Ginneken': {'ids': [8038506, 123637526],
  'years': {'start': 1996, 'end': 9999}},
 'Francesco Ciompi': {'ids': [143613202],
  'years': {'start': 2013, 'end': 9999}},
 'Alessa Hering': {'ids': [153744566], 'years': {'start': 2018, 'end': 9999}},
 'Henjan Huisman': {'ids': [34754023], 'years': {'start': 1992, 'end': 9999}},
 'Colin Jacobs': {'ids': [2895994], 'years': {'start': 2010, 'end': 9999}},
 'Peter Koopmans': {'ids': [34726383], 'years': {'start': 2022, 'end': 9999}},
 'Jeroen van der Laak': {'ids': [145441238, 145388932],
  'years': {'start': 1991, 'end': 9999}},
 'Geert Litjens': {'ids': [145959882], 'years': {'start': 2016, 'end': 9999}},
 'James Meakin': {'ids': [4960344], 'years': {'start': 2017, 'end': 9999}},
 'Keelin Murphy': {'ids': [35730362], 'years': {'start': 2018, 'end': 9999}},
 'Ajay Patel': {'ids': [2109170880, 2116215861],
  'years': {'start': 2015, 'end': 9999}},
 'Cornelia Schaefer-Prokop': {'ids': [1419819133, 1445069528, 1400632685],
  'years': {

In [31]:
# staff_ids = []
# [staff_ids.extend(values) for values in staff_dict.values()];
# staff_ids

## Semantic scolar

In [32]:
all_staff_id_ss_data = []

# get data for staff memb|er
for idx, (staff_name, values) in enumerate(staff_dict.items()):
    staff_ids = values['ids']
    staff_start = values['years']['start']
    staff_end = values['years']['end']
    print(f'[{idx+1}/{len(staff_dict.items())}]: {staff_name}')
    for staff_id in staff_ids:
        print('\t\t', staff_id)
        staff_id_ss_data = []
        # get semantic scolar data
        url=f'https://api.semanticscholar.org/graph/v1/author/{staff_id}/papers?fields=year,title,authors,externalIds,citationCount&limit=500'
        r = requests.get(url)
        ss_staff_data = r.json()['data']

        # loop over semantic scolar data entries
        for ss_staff_entry in ss_staff_data:
            ss_id = None
            ss_title = None
            ss_doi = None
            ss_citations = None
            ss_year = None

            if 'paperId' in ss_staff_entry:
                ss_id = ss_staff_entry['paperId']
            if 'title' in ss_staff_entry:
                ss_title = ss_staff_entry['title']
            if 'DOI' in ss_staff_entry['externalIds']:
                ss_doi = ss_staff_entry['externalIds']['DOI']
            if 'citationCount' in ss_staff_entry:
                ss_citations = ss_staff_entry['citationCount']
            if 'year' in ss_staff_entry:
                ss_year = ss_staff_entry['year']

            if ss_year != None:
                if not staff_start <= ss_year <= staff_end:
                # probably doesnt belong to DIAG, still captured via another staff member if also in the same paper
                    continue
                
            staff_id_ss_data.append([staff_id, staff_name, staff_start, staff_end, ss_year, ss_id, ss_title, ss_doi, ss_citations])
        all_staff_id_ss_data.extend(staff_id_ss_data)    
ss_columns = ['staff_id', 'staff_name', 'staff_from', 'staff_till', 'ss_year', 'ss_id', 'title', 'doi', 'ss_citations']
df_all_staff_id_ss_data = pd.DataFrame(all_staff_id_ss_data, columns=ss_columns)
print('DONE')

[1/20]: Bram van Ginneken
		 8038506
		 123637526
[2/20]: Francesco Ciompi
		 143613202
[3/20]: Alessa Hering
		 153744566
[4/20]: Henjan Huisman
		 34754023
[5/20]: Colin Jacobs
		 2895994
[6/20]: Peter Koopmans
		 34726383
[7/20]: Jeroen van der Laak
		 145441238
		 145388932
[8/20]: Geert Litjens
		 145959882
[9/20]: James Meakin
		 4960344
[10/20]: Keelin Murphy
		 35730362
[11/20]: Ajay Patel
		 2109170880
		 2116215861
[12/20]: Cornelia Schaefer-Prokop
		 1419819133
		 1445069528
		 1400632685
[13/20]: Matthieu Rutten
		 2074975080
		 2156546
[14/20]: Jos Thannhauser
		 5752941
[15/20]: Bram Platel
		 1798137
[16/20]: Nico Karssemeijer
		 1745574
[17/20]: Clarisa Sanchez
		 144085811
		 32187701
[18/20]: Nikolas Lessman
		 2913408
[19/20]: Jonas Teuwen
		 32649341
		 119024451
[20/20]: Rashindra Manniesing
		 2657081
DONE


In [33]:
df_all_staff_id_ss_data = df_all_staff_id_ss_data.sort_values('ss_id').reset_index(drop=True)

In [34]:
df_all_staff_id_ss_data.to_csv('script_data/temp_semantic_scholar_all_staff_id_last_500_summary.csv', index=False)

# Loop for matching DOIs and titles

In [35]:
from collections import Counter

In [43]:
doi_match_data = []
match_keys_columns = ['bibkey', 'ss_id', 'all_ss_ids', 'comment']
print('[MATCHING ON DOIS]')
# LOOP OVER BIB
for idx_bib, row_bib in df_bib_data.iterrows():
    bibkey = row_bib['bibkey']
    comment = None
    all_ss_ids = None
    
    bib_doi = row_bib['doi'].lower() if isinstance(row_bib['doi'], str) else row_bib['doi']
    if bib_doi == None:
        continue
    index = bib_doi.find('10.')
    bib_doi = bib_doi[index:] # remove url leading to doi identifier 
    doi_matching = df_all_staff_id_ss_data['doi'].str.lower().isin([bib_doi])
    
    if doi_matching.any():
        matches = df_all_staff_id_ss_data[doi_matching] # could be multiple in SS, search for "Epithelium segmentation using deep learning" paper from Wouter Bulten... 
        if len(Counter(matches['ss_id'])) == 1:
            comment = 'single doi match'
            match = matches.iloc[0]
        if len(Counter(matches['ss_id'])) > 1:
            all_ss_ids = list(Counter(matches["ss_id"]).keys())
            comment = f'multiple doi matches {all_ss_ids}, took the first one with most citations'
            match = matches[matches['ss_citations'] == matches['ss_citations'].max()].iloc[0]
        ss_id = match['ss_id']
        doi_match_data.append([bibkey, ss_id, all_ss_ids, comment])

df_doi_match_data = pd.DataFrame(doi_match_data, columns = match_keys_columns)
df_doi_match_data.to_csv('script_data/temp_doi_match_data.csv', index=False)
print('DONE')
print(len(df_doi_match_data['comment'].notna()))
print(len(df_doi_match_data))

[MATCHING ON DOIS]
DONE
898
898


In [44]:
import time

In [45]:
start = time.time()
print('[MATCHING ON TITLES]')
title_match_data = []
for idx_ss, row_ss in df_all_staff_id_ss_data.iterrows():
    if idx_ss % 50 == 0:
        print(f'[{idx_ss}/{len(df_all_staff_id_ss_data)}]')
    
    ss_id = row_ss['ss_id']
    # if ss_id in df_doi_match_data['ss_id']:
    #     continue
 
    ss_title = row_ss['title']
    if pd.isna(ss_title):
        continue
    
    title_match_ratios = df_bib_data_with_titles['title'].apply(lambda x: SequenceMatcher(
        a=ss_title.lower(), 
        b=x.lower().replace('{', '').replace('}', '')).ratio())

    max_ratio = title_match_ratios.max()
    max_bibkey = df_bib_data_with_titles[title_match_ratios==max_ratio]['bibkey'].iloc[0]
    max_bib_title = df_bib_data_with_titles[title_match_ratios==max_ratio]['title'].iloc[0]
    max_bib_title = max_bib_title.replace('{', '').replace('}', '')
    
    up80_bibkeys = None
    up80_bib_titles = None
    up80_ratios = None
    if sum(title_match_ratios>0.8) > 1:
        
        up80_bibkeys = list(df_bib_data_with_titles[title_match_ratios>0.8]['bibkey'])
        up80_bib_titles = list(df_bib_data_with_titles[title_match_ratios>0.8]['title'])
        up80_bib_titles = [up80_bib_title.replace('{', '').replace('}', '') for up80_bib_title in up80_bib_titles]
        up80_ratios = list(title_match_ratios[title_match_ratios>0.8])
    
    title_match_data.append([ss_id, ss_title, 
                             max_bibkey, max_bib_title, max_ratio, 
                             up80_bibkeys, up80_bib_titles, up80_ratios])

title_matches_columns = ['ss_id', 'ss_title', 
                         'max_bibkey' , 'max_bib_title', 'max_ratio',
                         'up80_bibkeys' , 'up80_bib_titles', 'up80_ratios',
                        ]
df_title_match_data = pd.DataFrame(title_match_data, columns=title_matches_columns)
df_title_match_data.to_csv('script_data/temp_title_match_data.csv', index=False)
print('DONE')
end = time.time()
sec = end-start
print(sec, 'sec')
print(sec/60, 'min')

[MATCHING ON TITLES]
[0/2202]
[50/2202]
[100/2202]
[150/2202]
[200/2202]
[250/2202]
[300/2202]
[350/2202]
[400/2202]
[450/2202]
[500/2202]
[550/2202]
[600/2202]
[650/2202]
[700/2202]
[750/2202]
[800/2202]
[850/2202]
[900/2202]
[950/2202]
[1000/2202]
[1050/2202]
[1100/2202]
[1150/2202]
[1200/2202]
[1250/2202]
[1300/2202]
[1350/2202]
[1400/2202]
[1450/2202]
[1500/2202]
[1550/2202]
[1600/2202]
[1650/2202]
[1700/2202]
[1750/2202]
[1800/2202]
[1850/2202]
[1900/2202]
[1950/2202]
[2000/2202]
[2050/2202]
[2100/2202]
[2150/2202]
[2200/2202]
DONE
1531.444974899292 sec
25.5240829149882 min


In [ ]:
print('DONE WITH THIS NOTEBOOK')
1/0

In [49]:
for entry in diag_bib_raw:
    if entry.key == 'Loog06a':
        print(entry.type)

article


In [50]:
for entry in diag_bib_raw:
    if entry.key == 'Loog03':
        print(entry.type)

inproceedings


In [52]:
for entry in diag_bib_raw:
    if entry.key == 'Moor18a':
        print(entry.type)

inproceedings


In [51]:
for entry in diag_bib_raw:
    if entry.key == 'Moor2018':
        print(entry.type)

article


In [53]:
for entry in diag_bib_raw:
    if entry.key == 'Anto21':
        print(entry.type)

article


In [55]:
for entry in diag_bib_raw:
    if entry.key == 'Anto22':
        print(entry.type)

article


In [57]:
for entry in diag_bib_raw:
    if entry.key == 'Wand16':
        print(entry.type)

conference


In [58]:
for entry in diag_bib_raw:
    if entry.key == 'Wand15b':
        print(entry.type)

conference


In [56]:
for entry in diag_bib_raw:
    if entry.key == 'Wand17a':
        print(entry.type)

article


# DONE